In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from keras import models, layers
from keras.layers import DepthwiseConv2D,SeparableConv2D,Conv2D,MaxPool2D,Dense,GlobalMaxPool2D,Flatten,Input,Add,BatchNormalization,GlobalAveragePooling2D,ReLU,Dropout,AveragePooling2D
import warnings
from sklearn import metrics
warnings.filterwarnings("ignore")
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.utils import to_categorical

In [ ]:
plt.figure(figsize=(20,40))
plt.subplot(2,2,1)

plt.imshow(cv2.imread("/kaggle/input/alaska2-image-steganalysis/Cover/00001.jpg"))
plt.title("Cover")


plt.subplot(2,2,2)
plt.imshow(cv2.imread("/kaggle/input/alaska2-image-steganalysis/JMiPOD/00001.jpg"))
plt.title("JMIPOd")


plt.subplot(2,2,3)
plt.imshow(cv2.imread("/kaggle/input/alaska2-image-steganalysis/JUNIWARD/00001.jpg"))
plt.title("JUNIWARD")


plt.subplot(2,2,4)
plt.imshow(cv2.imread("/kaggle/input/alaska2-image-steganalysis/UERD/00001.jpg"))
plt.title("UERD")

In [ ]:
def image_difference(path):
    image1=cv2.imread("/kaggle/input/alaska2-image-steganalysis/Cover/"+path,cv2.COLOR_BGR2RGB)
    image2=cv2.imread("/kaggle/input/alaska2-image-steganalysis/JMiPOD/"+path,cv2.COLOR_BGR2RGB)
    image3=cv2.imread("/kaggle/input/alaska2-image-steganalysis/JUNIWARD/"+path,cv2.COLOR_BGR2RGB)
    image4=cv2.imread("/kaggle/input/alaska2-image-steganalysis/UERD/"+path,cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(40,40))
    for j,i in enumerate([image2,image3,image4]):
        diff=np.abs(image1.astype(int)-i.astype(int)).astype(np.uint8)
        plt.subplot(2,2,j+1)
        plt.imshow(diff[:,:,0],vmin=0,vmax=np.amax(diff[:,:,0]),cmap="hot")

Darker Regions:

Darker regions in the image, represented by black, indicate areas where the pixel values have smaller differences between the two images.
Warmer Colors:

Warmer colors such as red and yellow indicate areas of higher intensity or larger differences between the two images.
Intensity Gradient:

The intensity of the colors (from black to red to yellow) represents the magnitude of the differences. Darker colors suggest small differences, while brighter colors suggest larger differences.
Spotting Anomalies:

Look for anomalies or distinct features in the image. Hotter colors can highlight specific areas where changes or discrepancies occur between the images.

In [ ]:
def channel_wise_difference(path):
    image1=cv2.imread("/kaggle/input/alaska2-image-steganalysis/Cover/"+path,cv2.COLOR_BGR2RGB)
    image2=cv2.imread("/kaggle/input/alaska2-image-steganalysis/JMiPOD/"+path,cv2.COLOR_BGR2RGB)
    image3=cv2.imread("/kaggle/input/alaska2-image-steganalysis/JUNIWARD/"+path,cv2.COLOR_BGR2RGB)
    image4=cv2.imread("/kaggle/input/alaska2-image-steganalysis/UERD/"+path,cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(nrows=3, ncols=4, figsize=(16, 12))
    ax[0,0].imshow(image2)
    ax[0,1].imshow((image1 == image2).astype(int)[:,:,0])
    ax[0,1].set_title(f' Channel 0')

    ax[0,2].imshow((image1 == image2).astype(int)[:,:,1])
    ax[0,2].set_title(f' Channel 1')
    ax[0,3].imshow((image1 == image2).astype(int)[:,:,2])
    ax[0,3].set_title(f' Channel 2')
    ax[0,0].set_ylabel('JMiPOD', rotation=90, fontsize=14)


    ax[1,0].imshow(image3)
    ax[1,1].imshow((image1 == image3).astype(int)[:,:,0])
    ax[1,2].imshow((image1 == image3).astype(int)[:,:,1])
    ax[1,3].imshow((image1 == image3).astype(int)[:,:,2])
    ax[1,0].set_ylabel('JUNIWARD', rotation=90, fontsize=14)

    ax[2,0].imshow(image4)
    ax[2,1].imshow((image1 == image4).astype(int)[:,:,0])
    ax[2,2].imshow((image1 == image4).astype(int)[:,:,1])
    ax[2,3].imshow((image1 == image4).astype(int)[:,:,2])
    ax[2,0].set_ylabel('UERD', rotation=90,  fontsize=14)

    plt.suptitle('Pixel Deviation from Cover Image', fontsize=14)

    plt.show()

In [ ]:
def hist_image_difference(path):
    image1=cv2.imread("/kaggle/input/alaska2-image-steganalysis/Cover/"+path,cv2.COLOR_BGR2RGB)
    image2=cv2.imread("/kaggle/input/alaska2-image-steganalysis/JMiPOD/"+path,cv2.COLOR_BGR2RGB)
    image3=cv2.imread("/kaggle/input/alaska2-image-steganalysis/JUNIWARD/"+path,cv2.COLOR_BGR2RGB)
    image4=cv2.imread("/kaggle/input/alaska2-image-steganalysis/UERD/"+path,cv2.COLOR_BGR2RGB)
    algorithm = ('Cover(Unaltered)', 'JMiPOD', 'UERD', 'JUNIWARD') 
    cover_hist = {}
    jmipod_hist = {}
    uerd_hist = {}
    juniward_hist = {}


    color = ('b', 'g', 'r')


    for i, col in enumerate(color):

        cover_hist[col] = cv2.calcHist([image1], [i], None, [256], [0, 256])
        jmipod_hist[col] = cv2.calcHist([image2], [i], None, [256], [0, 256])
        uerd_hist[col] = cv2.calcHist([image3], [i], None, [256], [0, 256])
        juniward_hist[col] = cv2.calcHist([image4], [i], None, [256], [0, 256])
    fig_hist, axes_hist = plt.subplots(nrows=2, ncols=2, figsize=(12, 12))


    for ax, hist, algo in zip(axes_hist.flat, [cover_hist, jmipod_hist, uerd_hist, juniward_hist], algorithm):
        ax.plot(hist['r'], color='r', label='r')  
        ax.plot(hist['g'], color='g', label='g')  
        ax.plot(hist['b'], color='b', label='b')  
        ax.set(ylabel='# of pixels', xlabel='Pixel value (0-255)', title=algo)
        ax.legend()


    fig_hist.subplots_adjust(wspace=0.4, hspace=0.3)


    fig_hist.suptitle('Histogram of a sample 1.jpg', fontsize=20)


    plt.show()

In [ ]:
image_difference("00002.jpg")

Darker Regions:

Darker regions in the image, represented by black, indicate areas where the pixel values have smaller differences between the two images.
Warmer Colors:

Warmer colors such as red and yellow indicate areas of higher intensity or larger differences between the two images.
Intensity Gradient:

The intensity of the colors (from black to red to yellow) represents the magnitude of the differences. Darker colors suggest small differences, while brighter colors suggest larger differences.
Spotting Anomalies:

Look for anomalies or distinct features in the image. Hotter colors can highlight specific areas where changes or discrepancies occur between the images.

In [ ]:
channel_wise_difference("00002.jpg")

In [ ]:
hist_image_difference("00002.jpg")